<a href="https://colab.research.google.com/github/ssonone/FutureTechTrajectories/blob/main/GlobalRndServerDownload_largedata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import Modules
import os
import pandas as pd
from sqlalchemy import create_engine
from tqdm.auto import tqdm
import pymysql
from dotenv import load_dotenv

# pymysql을 MySQLdb로 사용하기 위한 설정
pymysql.install_as_MySQLdb()


# 환경변수 파일 로드
load_dotenv()

True

In [ ]:
def get_database_engine():
    # 환경변수에서 데이터베이스 연결 정보 가져오기
    user = os.getenv('DB_USER')
    pw = os.getenv('DB_PASSWORD')
    ip = os.getenv('DB_IP')
    port = os.getenv('DB_PORT')
    db = os.getenv('DB_NAME')
    return create_engine(f"mysql://{user}:{pw}@{ip}:{port}/{db}")

In [ ]:
def fetch_data(engine, year, table_name):
    query = f"""
    SELECT B.* FROM `web_of_science_2413_raw`.`{table_name}` B
    JOIN `web_of_science_2413_raw`.`pub_info` A ON A.uid = B.uid
    WHERE A.pubyear = %s;
    """
    return pd.read_sql_query(query, engine, params=[(year,)])

In [ ]:
## 데이터가 큰 경우
def main():
    engine = get_database_engine()
    dataframes = []

    # 데이터베이스 이름 환경변수에서 불러오기
    db_name = os.getenv('DB_NAME')

    # 사용자로부터 처리할 테이블 이름 입력 받기
    table_name = input("Enter the table name to process (e.g., title, abstract, keyword): ")

    # 시작 연도 설정
    start_year = 2006
    end_year = 2023

    # 진행 상태를 확인하고, 중단된 연도부터 재시작
    for year in tqdm(range(start_year, end_year + 1), desc=f"processing {table_name}"):
        file_path = f'yearly_data/{db_name}_{table_name}_{year}.csv'

        # 파일이 이미 존재하면 스킵 (재처리 방지)
        if os.path.exists(file_path):
            print(f"Skipping {year}, file already exists.")
            continue

        try:
            # 데이터 가져오기
            df = fetch_data(engine, year, table_name)
            # 연도별 파일로 CSV 형태로 저장
            df.to_csv(file_path, index=False)
            print(f"Data for {year} saved successfully.")
        except Exception as e:
            print(f"Failed to fetch data for {year}. Error: {e}")
            break  # 오류 발생 시 중단

    # 모든 연도 파일 로드 및 합치기
    dataframes = []
    for year in range(start_year, end_year + 1):
        file_path = f'yearly_data/{db_name}_{table_name}_{year}.csv'
        if os.path.exists(file_path):
            df = pd.read_csv(file_path)
            dataframes.append(df)

    combined_df = pd.concat(dataframes, ignore_index=True)
    combined_df.to_pickle(f'combined_df_{db_name}_{table_name}.pkl')

    engine.dispose()

if __name__ == "__main__":
    main()


Enter the table name to process (e.g., title, abstract, keyword): author_total


processing author_total:   0%|          | 0/18 [00:00<?, ?it/s]

Skipping 2006, file already exists.
Skipping 2007, file already exists.
Skipping 2008, file already exists.
Skipping 2009, file already exists.
Skipping 2010, file already exists.
Skipping 2011, file already exists.
Skipping 2012, file already exists.
Skipping 2013, file already exists.
Skipping 2014, file already exists.
Skipping 2015, file already exists.
Skipping 2016, file already exists.
Skipping 2017, file already exists.
Skipping 2018, file already exists.
Skipping 2019, file already exists.
Data for 2020 saved successfully.
Data for 2021 saved successfully.
Data for 2022 saved successfully.
Data for 2023 saved successfully.


C:\Users\skim\AppData\Local\Temp\ipykernel_9000\3921889569.py:40: DtypeWarning: Columns (6,8,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\skim\AppData\Local\Temp\ipykernel_9000\3921889569.py:40: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\skim\AppData\Local\Temp\ipykernel_9000\3921889569.py:40: DtypeWarning: Columns (6,7,8,10,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\skim\AppData\Local\Temp\ipykernel_9000\3921889569.py:40: DtypeWarning: Columns (6,8,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\skim\AppData\Local\Temp\ipykernel_9000\3921889569.py:40: DtypeWarning: Columns (6,7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fi